# Analyse du texte

## Charger les données

In [1]:
import pandas as pd
url = "https://zenodo.org/record/5827206/files/SOSP_Export_base%20de%20donn%C3%A9es%20diffusable.csv"
data = pd.read_csv(url)
data.head()

,Id,contexte_travail,fonction_recherche,statut_professionnel,statut_professionnel_Autre,annee_premiere_publi,Systeme_exploitation,Usage_telephone_mobile,Outils_gestion_travail_av_confinement_visioconférence,Outils_gestion_travail_av_confinement_planification_RDV,...,Souhait_partage_donnees_produites_discipline,Souhait_partage_donnees_produites_pays_continent,Souhait_partage_donnees_produites_domaine_economique,Souhait_partage_donnees_produites_secteur_associatif,Souhait_partage_donnees_produites_sans_restriction,compatibilite_diffusion_partage,evolutions_pratiques_numeriques,sexe,disciplines_9niv,annee_de_naissance-recod10niv
0,1,nous sommes entre 6 et 10 personnes,professeur / professeure des universités et as...,fonctionnaire ou assimilé,NaN,1995,Windows,rarement,parfois,souvent,...,oui,oui,oui,oui,oui,oui,je ne sais pas,un homme,Sciences humaines,61 à 65 ans
1,2,nous sommes plus de 50 personnes,ingénieur / ingénieure de recherche,en CDI,NaN,2005,Windows,rarement,parfois,parfois,...,oui,oui,oui,oui,oui,NaN,NaN,un homme,Médecine,46 à 50 ans
2,3,nous sommes entre 2 et 5 personnes,maître / maîtresse de conférences et assimilés,fonctionnaire ou assimilé,NaN,2006,Windows,parfois,rarement,souvent,...,oui,oui,oui,oui,oui,"oui, tout est dans la chronologie",non,un homme,Médecine,36 à 40 ans
3,4,nous sommes entre 2 et 5 personnes,"directeur / directrice de recherche (CNRS, INS...",fonctionnaire ou assimilé,NaN,1989,Windows;autres OS (Android...),souvent,rarement,souvent,...,oui,oui,non,non,non,Oui,Aucune idée,une femme,Médecine,56 à 60 ans
4,5,nous sommes entre 2 et 5 personnes,doctorant / doctorante (y compris CIFRE),en CDD,NaN,2015,MacOS,souvent,rarement,parfois,...,oui,oui,oui,oui,oui,Oui,Oui,une femme,Médecine,31 à 35 ans


Variables qualitatives textuelles que je veux analyser

- ["logiciel_production_donnees","outils_nettoyage","Outils_analyses","Outils_visualisation"]
- evolutions_pratiques_numeriques

Jeter un coup d'oeil

In [9]:
#list(data["logiciel_production_donnees"].dropna())

Les étapes d'une analyses textuelle :

- nettoyer les données en amont de l'analyse pour avoir des textes sous format numérique
- tokenization 
- enlever du contenu non signiant : "stop word"  plus corriger
- lemmatisation 
- tableau documents vs termes
- traitement statistique et Machine Learning : cluster, compter des fréquences, etc.

# Approche des sacs de mots

Chaque étape du traitement peut être modifié et avoir des conséquences sur les résultats

In [18]:
data[colonnes]

,logiciel_production_donnees,outils_nettoyage,Outils_analyses,Outils_visualisation
0,Lime survey,NaN,tableur pour les données statistiques,NaN
1,excell word,clean,outil statistique,NaN
2,JASP,NaN,NaN,NaN
3,"Question peu claire. Excel, SPSS, logiciels d...",NaN,"Excel, SPSS",NaN
4,?,NaN,Excel r,Image j dti studii
...,...,...,...,...
1084,NaN,NaN,EpiInfo\nXlstat,EpiInfo\nXlstat\nExcel
1085,NaN,NaN,NaN,"Xpdf, Evince"
1086,NaN,"openrefine, talend",NaN,NaN
1087,NaN,NaN,NaN,NaN


Réunir les textes ensemble

In [30]:
def creation_texte(ligne):
    colonnes = ["logiciel_production_donnees","outils_nettoyage","Outils_analyses","Outils_visualisation"]
    return " ".join([texte for texte in ligne[colonnes] if pd.notna(texte)])

data["texte"] = data[colonnes].apply(creation_texte,axis=1)

Joindre chaine de caractères

In [16]:
"-".join(["a","b"])

'a-b'

Compréhension de liste

In [21]:
nouvelle_liste = []
for element in ["a","b","c"]:
    nouvelle_liste.append(element)

In [23]:
nouvelle_liste = [element for element in ["a","b","c"]]

In [24]:
{e:e for e in ["a","b","b"]}

{'a': 'a', 'b': 'b'}

In [27]:
[element for element in ["a","b","c",None] if element != None]

['a', 'b', 'c']

Fonction anonyme

In [32]:
(lambda x : x +1)(10)

11

Recodage du texte en une ligne

In [35]:
data["texte"] = data[colonnes].apply(lambda x : " ".join([texte for texte in x if pd.notnull(texte)]),axis=1)

### Tokénisons le texte

Séparer en mots, enlever les ponctuations, enlever les mots inutiles

In [64]:
def tokeniser_texte(texte):
    seps = [" ",",","'",":",".","\n","\r","?"] # définir les séparateurs
    stopwords = ["de","la","le","les","a","autre",
                 "où","","et","ou","en","des","d","je"] # définit les stopwords
    texte_tokenize = texte.lower()
    # marquer les séparateurs de la même manière
    for s in seps:
        texte_tokenize = texte_tokenize.replace(s," ")
    # couper au niveau des séparateurs
    texte_tokenize = texte_tokenize.split(" ")
    
    # enlever les stopwords
    texte_tokenize = [m for m in texte_tokenize if not m in stopwords]
    
    # retourner le sac de mots
    return texte_tokenize

In [65]:
data["texte_tokenize"] = data["texte"].apply(tokeniser_texte)

### On peut faire du comptage

In [60]:
from collections import Counter

In [66]:
Counter(data["texte_tokenize"].iloc[3])

Counter({'question': 1,
         'peu': 1,
         'claire': 1,
         'excel': 2,
         'spss': 2,
         'logiciels': 1,
         'imagerie': 1,
         'commerciaux': 1,
         '(philips)': 1})

In [67]:
all_words = []
for t in list(data["texte_tokenize"]):
    all_words+=t
len(all_words)

11475

In [68]:
Counter(all_words).most_common(20)

[('excel', 520),
 ('r', 511),
 ('python', 327),
 ('logiciels', 236),
 ('matlab', 219),
 ('word', 157),
 ('données', 154),
 ('-', 119),
 ('qgis', 106),
 ('outils', 95),
 ('pour', 94),
 ('logiciel', 92),
 ('office', 91),
 ('par', 81),
 ('spss', 80),
 (';', 76),
 ('pas', 74),
 ('imagej', 67),
 ('sas', 66),
 ('à', 60)]

### Construire la matrice documents (ligne) et les N premiers termes (en colonne)

In [74]:
ref = [i[0] for i in Counter(all_words).most_common(100)]
matrice = []
for doc in list(data["texte_tokenize"]):
    ligne = []
    for m in ref:
        ligne.append(doc.count(m))
    matrice.append(ligne)

Le tableau numérique du texte

In [78]:
matrice = pd.DataFrame(matrice,columns=ref)

La porte est ouverte aux traitements plus avancés

On a envie de faire une analyse en composante principale (ACP)

In [81]:
import prince

In [82]:
acp = prince.PCA()

In [83]:
acp = acp.fit(matrice)

In [84]:
acp.plot(matrice)

alt.LayerChart(...)